# ЛР8. Алгоритм Витерби для HMM

## 1. Исходные данные (считано с рисунка)

### 1.1. Скрытые состояния (Hidden states)
Обозначим три рынка:
- **F** — Food street  
- **C** — Clothes market  
- **E** — Electronics market  

Порядок состояний в дальнейших матрицах фиксируем как: **[F, C, E]**.

---

### 1.2. Наблюдения (Observations)
Возможные наблюдаемые категории:
- **Food**
- **Clothes**
- **Electronics**

Порядок наблюдений в матрице эмиссий: **[Food, Clothes, Electronics]**.

---

### 1.3. Начальные вероятности π
Согласно рисунку, стартовая вероятность для каждого скрытого состояния одинакова:

\[
\pi(F)=0.33,\quad \pi(C)=0.33,\quad \pi(E)=0.33
\]

Вектор:

\[
\pi = [0.33,\;0.33,\;0.33]
\]

> Примечание: сумма 0.99 — это округление на схеме, оставляем как задано.

---

### 1.4. Матрица переходов A (Hidden → Hidden)
Переходы между рынками:

- из **F**: \(F\to F=0.10,\;F\to C=0.60,\;F\to E=0.30\)
- из **C**: \(C\to F=0.45,\;C\to C=0.15,\;C\to E=0.40\)
- из **E**: \(E\to F=0.40,\;E\to C=0.40,\;E\to E=0.20\)

Матрица \(A\) (строки — откуда, столбцы — куда; порядок [F,C,E]):

\[
A =
\begin{pmatrix}
0.10 & 0.60 & 0.30 \\
0.45 & 0.15 & 0.40 \\
0.40 & 0.40 & 0.20
\end{pmatrix}
\]

---

### 1.5. Матрица эмиссий B (Hidden → Observation)
Вероятности “что увидим”, если находимся в конкретном скрытом состоянии:

- для **F**: \(P(Food)=0.70,\;P(Clothes)=0.20,\;P(Electronics)=0.10\)
- для **C**: \(P(Food)=0.25,\;P(Clothes)=0.65,\;P(Electronics)=0.10\)
- для **E**: \(P(Food)=0.15,\;P(Clothes)=0.10,\;P(Electronics)=0.75\)

Матрица \(B\) (порядок наблюдений [Food, Clothes, Electronics]):

\[
B =
\begin{pmatrix}
0.70 & 0.20 & 0.10 \\
0.25 & 0.65 & 0.10 \\
0.15 & 0.10 & 0.75
\end{pmatrix}
\]

---

## 2. Последовательность наблюдений
Берём стандартную длину 3 (как в примере из статьи):

\[
O = [Food,\;Clothes,\;Electronics]
\]

In [1]:
import numpy as np
import pandas as pd

# 1) Справочники (чтобы дальше было удобно работать и красиво печатать)
STATE_ORDER = ("F", "C", "E")
OBS_ORDER = ("Food", "Clothes", "Electronics")

state_name = {
    "F": "Food street (F)",
    "C": "Clothes market (C)",
    "E": "Electronics market (E)",
}

# 2) Параметры HMM (те же числа, просто оформлены иначе)
pi = np.array([1/3, 1/3, 1/3]) * 0.99  # чтобы получилось [0.33, 0.33, 0.33] как на рисунке
# если хочешь строго ровно как у тебя:
# pi = np.array([0.33, 0.33, 0.33], dtype=float)

A = pd.DataFrame(
    {
        "F": [0.10, 0.45, 0.40],
        "C": [0.60, 0.15, 0.40],
        "E": [0.30, 0.40, 0.20],
    },
    index=["F", "C", "E"],
).to_numpy(dtype=float)

B = pd.DataFrame(
    {
        "Food": [0.70, 0.25, 0.15],
        "Clothes": [0.20, 0.65, 0.10],
        "Electronics": [0.10, 0.10, 0.75],
    },
    index=["F", "C", "E"],
).to_numpy(dtype=float)

# 3) Последовательность наблюдений (та же, но индексируем через словарь)
O = ("Food", "Clothes", "Electronics")
obs_to_idx = {obs: i for i, obs in enumerate(OBS_ORDER)}
O_idx = [obs_to_idx[x] for x in O]

# 4) Возврат/проверка
states = [state_name[s] for s in STATE_ORDER]
observations = list(OBS_ORDER)

pi, A, B, O_idx

(array([0.33, 0.33, 0.33]),
 array([[0.1 , 0.6 , 0.3 ],
        [0.45, 0.15, 0.4 ],
        [0.4 , 0.4 , 0.2 ]]),
 array([[0.7 , 0.2 , 0.1 ],
        [0.25, 0.65, 0.1 ],
        [0.15, 0.1 , 0.75]]),
 [0, 1, 2])

In [2]:
import numpy as np
import pandas as pd

T = len(O_idx)
N = len(states)

# --- Витерби (та же логика, но иначе написано) ---

delta = np.zeros((T, N), dtype=float)          # Viterbi scores
psi = np.full((T, N), -1, dtype=int)           # backpointers

# t = 0: инициализация одной строкой
delta[0] = pi * B[:, O_idx[0]]

# t = 1..T-1: векторно по всем "откуда" для каждого j
for t in range(1, T):
    # prev[:, None] * A -> матрица (i -> j): вероятность прийти в j из i
    trans_scores = delta[t - 1][:, None] * A

    # для каждого j выбираем лучшее i
    psi[t] = np.argmax(trans_scores, axis=0)

    # delta[t, j] = max_i(...) * B[j, obs_t]
    delta[t] = trans_scores[psi[t], np.arange(N)] * B[:, O_idx[t]]

# --- Вывод таблиц ---
delta_df = pd.DataFrame(delta, columns=states, index=[f"t={t}" for t in range(T)])
psi_df = pd.DataFrame(psi, columns=states, index=[f"t={t}" for t in range(T)])

print("Наблюдения O =", list(O))
print("\nDELTA (вероятности):")
display(delta_df)

print("\nPSI (указатели откуда пришли, индексы состояний):")
display(psi_df)

# --- Финал: лучший последний стейт и восстановление пути ---
last_state = int(delta[-1].argmax())
best_prob = float(delta[-1, last_state])

path_idx = np.empty(T, dtype=int)
path_idx[-1] = last_state

for t in range(T - 1, 0, -1):
    path_idx[t - 1] = psi[t, path_idx[t]]

best_path = [states[i] for i in path_idx]

print("\nЛУЧШАЯ СКРЫТАЯ ПОСЛЕДОВАТЕЛЬНОСТЬ:")
for t, s in enumerate(best_path):
    print(f"t={t}, obs={O[t]}  ->  hidden={s}")

print(f"\nМаксимальная вероятность пути P* = {best_prob}")

Наблюдения O = ['Food', 'Clothes', 'Electronics']

DELTA (вероятности):


,Food street (F),Clothes market (C),Electronics market (E)
t=0,0.231000,0.082500,0.049500
t=1,0.007425,0.090090,0.006930
t=2,0.004054,0.001351,0.027027



PSI (указатели откуда пришли, индексы состояний):


,Food street (F),Clothes market (C),Electronics market (E)
t=0,-1,-1,-1
t=1,1,0,0
t=2,1,1,1



ЛУЧШАЯ СКРЫТАЯ ПОСЛЕДОВАТЕЛЬНОСТЬ:
t=0, obs=Food  ->  hidden=Food street (F)
t=1, obs=Clothes  ->  hidden=Clothes market (C)
t=2, obs=Electronics  ->  hidden=Electronics market (E)

Максимальная вероятность пути P* = 0.027027
